## Build a baseline model

In [1]:
import pandas as pd
import numpy as np

To download and read dataset into dataframe

In [4]:
!wget https://cocl.us/concrete_data
!mv concrete_data concrete_data.csv
data=pd.read_csv('concrete_data.csv')
data.head()

--2020-11-03 08:13:49--  https://cocl.us/concrete_data
Resolving cocl.us (cocl.us)... 169.63.96.194, 169.63.96.176
Connecting to cocl.us (cocl.us)|169.63.96.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv [following]
--2020-11-03 08:13:50--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58988 (58K) [text/csv]
Saving to: ‘concrete_data’

concrete_data       100%[===================>]  57.61K  --.-KB/s    in 0.1s    

2020-11-03 08:13:51 (389 KB/s) - ‘concre

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.

In [5]:
data.shape

(1030, 9)

So, there are approximately 1030 samples to train model on.

To check dataset for missing values:

In [6]:
data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [8]:
concrete_data_columns=data.columns
predictors=data[concrete_data_columns[concrete_data_columns != 'Strength']] # All columns except Strength
target=data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [11]:
ncol=predictors.shape[1] # Total predictors
ncol

8

<a id="item1"></a>

<a id="item1"></a>

## Import Keras

In [2]:
import keras

In [3]:
from keras.models import Sequential
from keras.layers import Dense

To create model that has one hidden layer with 10 nodes and a ReLU activation function. It uses the adam optimizer and mean squared error as the loss function.

In [17]:
# define regression model
def regression_model():
    model = Sequential()
    model.add(Dense(10,activation='relu',input_shape=(ncol,)))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model

In [21]:
from sklearn.model_selection import train_test_split

Randomly split data into training and test sets by holding 30% of data for testing

In [22]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

## Train and Test the Network

Let's call the function now to create our model.

In [29]:
model=regression_model()

To train model on training data using 50 epochs:

In [30]:
epochs=50
model.fit(X_train,y_train,epochs=epochs,verbose=1)

Epoch 1/50
23/23 [==============================] - 0s 4ms/step - loss: 3439.3083
Epoch 2/50
23/23 [==============================] - 0s 2ms/step - loss: 2438.0864
Epoch 3/50
23/23 [==============================] - 0s 3ms/step - loss: 1855.9879
Epoch 4/50
23/23 [==============================] - 0s 3ms/step - loss: 1419.9175
Epoch 5/50
23/23 [==============================] - 0s 3ms/step - loss: 1061.1891
Epoch 6/50
23/23 [==============================] - 0s 2ms/step - loss: 820.5703
Epoch 7/50
23/23 [==============================] - 0s 2ms/step - loss: 669.7319
Epoch 8/50
23/23 [==============================] - 0s 2ms/step - loss: 575.3070
Epoch 9/50
23/23 [==============================] - 0s 2ms/step - loss: 496.9851
Epoch 10/50
23/23 [==============================] - 0s 2ms/step - loss: 431.5261
Epoch 11/50
23/23 [==============================] - 0s 2ms/step - loss: 386.1855
Epoch 12/50
23/23 [==============================] - 0s 2ms/step - loss: 350.4974
Epoch 13/50
23/23 [=

Next we need to evaluate the model on the test data.

In [33]:
loss_val=model.evaluate(X_test,y_test)
ypred=model.predict(X_test)
loss_val

10/10 [==============================] - 0s 2ms/step - loss: 116.8374


116.83744049072266

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [34]:
from sklearn.metrics import mean_squared_error

In [38]:
mse=mean_squared_error(y_test, ypred)
mean = np.mean(mse)
sd=np.std(mse)
print(mean,sd)

53.75094830108126 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [42]:
totalmse=50
epochs=50
mean_squared_errors=[]
for i in range(0, totalmse):
    X_train, X_test, y_train, y_test=train_test_split(predictors,target,test_size=0.3,random_state=i)
    model.fit(X_train, y_train,epochs=epochs,verbose=0)
    MSE = model.evaluate(X_test,y_test,verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mse)
mean_squared_errors=np.array(mean_squared_errors)
mean=np.mean(mean_squared_errors)
sd=np.std(mean_squared_errors)
print("\nMean and standard deviation of " +str(totalmse) + " mean squared errors without normalization:\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(sd))

MSE 1: 44.58778762817383
MSE 2: 45.33092498779297
MSE 3: 39.72268295288086
MSE 4: 42.366050720214844
MSE 5: 42.221343994140625
MSE 6: 45.200138092041016
MSE 7: 51.78047561645508
MSE 8: 38.537689208984375
MSE 9: 47.95219421386719
MSE 10: 43.836341857910156
MSE 11: 41.739933013916016
MSE 12: 40.59569549560547
MSE 13: 49.225589752197266
MSE 14: 49.78409957885742
MSE 15: 42.92987060546875
MSE 16: 35.45029067993164
MSE 17: 44.406410217285156
MSE 18: 46.145545959472656
MSE 19: 41.78548812866211
MSE 20: 45.60719299316406
MSE 21: 38.55569076538086
MSE 22: 40.87824249267578
MSE 23: 36.72555923461914
MSE 24: 38.83198928833008
MSE 25: 42.07720184326172
MSE 26: 44.45423889160156
MSE 27: 42.9725341796875
MSE 28: 40.211917877197266
MSE 29: 47.34975814819336
MSE 30: 43.849220275878906
MSE 31: 40.871768951416016
MSE 32: 37.550392150878906
MSE 33: 39.79094696044922
MSE 34: 44.049251556396484
MSE 35: 42.59197998046875
MSE 36: 47.60944366455078
MSE 37: 45.153175354003906
MSE 38: 46.77328109741211
MSE 39: